## Домашнее задание по неделе 4

Как было рассказано на лекции, стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

### Задание 0

Реализуйте класс ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [2]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon = 1e-4, max_steps = 100, w0=None, alpha = 1e-4):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape
        curindex = np.random.randint(0, l)
        ## На каждом шаге градиентного спуска веса необходимо добавлять в w_history
        if self.w0 is None:
          self.w0 = np.zeros(d)
        self.w = self.w0
        for step in range(self.max_steps):
          self.w_history.append(self.w)
          print(X[curindex])
          new_w = self.w - self.alpha * self.calc_gradient(X[curindex], y[curindex])

          if (np.linalg.norm(new_w - self.w) <=  self.epsilon):
            self.w = new_w
            break
          self.w = new_w
          curindex = np.random.randint(0, l)
        
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        
        return np.dot(X, self.w)
    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        grad = 2 * np.dot(X.T, np.dot(X, self.w - y))
        return grad

Проверять работу мы будем на имеющемся в sklearn наборе данных boston: в нём нужно по информации о доме предсказать его стоимость.

In [3]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size = 0.3, random_state = 10)

### Задание 1

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите 
```MAPE(y_test, y_0)```, где ```y_0 = (mean(y_test), mean(y_test), ...)```

In [4]:
def MAPE(y_true, y_pred):
    """
        y_true: np.array (l)
        y_pred: np.array (l)
        ---
        output: float [0, +inf)
    """
    return np.abs((y_true - y_pred)/y_true).mean() * 100

In [5]:
l = y_test.shape
y_0 = np.full(l, y_test.mean())
MAPE(y_test, y_0)

Result: 37.41588297684096

### Задание 2 

Обучите ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```), сделайте предсказание на тестовых данных ```X_test```, записав результат в переменную ```y_pred_sgd```  и вычислите ошибку MAPE.

In [11]:
sgd = LinearRegressionSGD().fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
MAPE(y_test, y_pred_sgd)

[1.1329e-01 3.0000e+01 4.9300e+00 0.0000e+00 4.2800e-01 6.8970e+00
 5.4300e+01 6.3361e+00 6.0000e+00 3.0000e+02 1.6600e+01 3.9125e+02
 1.1380e+01]
[3.3147e-01 0.0000e+00 6.2000e+00 0.0000e+00 5.0700e-01 8.2470e+00
 7.0400e+01 3.6519e+00 8.0000e+00 3.0700e+02 1.7400e+01 3.7895e+02
 3.9500e+00]
[2.6363e-01 0.0000e+00 8.5600e+00 0.0000e+00 5.2000e-01 6.2290e+00
 9.1200e+01 2.5451e+00 5.0000e+00 3.8400e+02 2.0900e+01 3.9123e+02
 1.5550e+01]
[5.3600e-02 2.1000e+01 5.6400e+00 0.0000e+00 4.3900e-01 6.5110e+00
 2.1100e+01 6.8147e+00 4.0000e+00 2.4300e+02 1.6800e+01 3.9690e+02
 5.2800e+00]
[2.0608e-01 2.2000e+01 5.8600e+00 0.0000e+00 4.3100e-01 5.5930e+00
 7.6500e+01 7.9549e+00 7.0000e+00 3.3000e+02 1.9100e+01 3.7249e+02
 1.2500e+01]
[3.1827e-01 0.0000e+00 9.9000e+00 0.0000e+00 5.4400e-01 5.9140e+00
 8.3200e+01 3.9986e+00 4.0000e+00 3.0400e+02 1.8400e+01 3.9070e+02
 1.8330e+01]
[  1.20742   0.       19.58      0.        0.605     5.875    94.6
   2.4259    5.      403.       14.7     292.29    

Result: 7.971608836356955e+179

### Задание 3

Вычислите веса по точной формуле, используя ```X_train``` и ```y_train```; предскажите с их помощью целевую переменную на ```X_test```, записав результат в переменную ```y_pred_lr``` и вычислите ошибку MAPE.

In [8]:
y_pred_lr = X_test @ ((np.linalg.inv(X_train.T @ X_train) @ X_train.T) @ y_train)
MAPE(y_test, y_pred_lr)

Result: 18.953134816375112

## Бонусное задание по неделе 4

До этого вы релизовывали модели, в которых не было штрафа за величину весов ```w```. Как было рассказано ранее в лекциях, это может привести к неустойчивости модели и переобучению. Чтобы избежать этих эффектов, предлагается добавить к оптимизируемому функционалу L2-норму весов; таким образом, будем решать задачу гребневой регрессии, Ridge:

$$ \frac{1}{l}(Xw-y)^T(Xw-y) +\gamma||w||_2 \rightarrow \min_{w}. $$

### Задание 11
Реализуйте обучение такой модели в матричном виде (смотрите дополнительные материалы к этой неделе) с помощью стохастического градиентного спуска. Класс должен совпадать по набору реализованных функций с ```LinearRegressionVectorized```, разница будет лишь в параметре $\gamma$, отвечающем за степень регуляризации. 

In [0]:
class RidgeSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2, gamma=0):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        gamma: коэффициент регуляризации
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.gamma = gamma
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        pass

    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        pass

Так же, как и в основном задании, обучите модель с базовыми параметрами на тренировочных данных и сделайте прогноз y_pred_ridge. Выведите значение MAPE(y_test, y_pred_ridge).